In [1]:
from utils.get_data import generate_data_nested_dirs, get_cube_at_point, generate_cubes, generate_cube_batch
import numpy as np

In [ ]:
def plot_one_scan(scan:np.array):
    plt.figure(figsize=(32,128))
    plt.gray()
    plt.subplots_adjust(0,0,0.5,0.5,0.01,0.01)
    for i in range(scan.shape[0]):
        shape0 = scan.shape[0] // 4 + 1
        plt.subplot(shape0, 4, i + 1)
        plt.imshow(scan[i])
        plt.axis('off')
    plt.show()

In [ ]:
def scan_images(scan:np.array, filename='scan_image', sep='_'):
    for i in range(scan.shape[0]):
        plt.gray()
        plt.figure(figsize=(10,10))
        plt.imshow(scan[i])
        plt.axis('off')
        plt.savefig(f'{filename}{sep}{i}.png', bbox_inches='tight', pad_inches=0, transparent=True)

In [2]:
import pandas as pd

df_annotations = pd.read_csv('CSVFILES/annotations.csv')

In [ ]:
scans = generate_data_nested_dirs('./data')

In [ ]:
cubes = generate_cubes(df_annotations, (64, 128, 128), root_dir='./data')

In [ ]:
plot_one_scan(next(scans)[0])

In [ ]:
# cube = next(cubes)
# scan_images(cube[0], 'cube')
# print(f'Tumor: {cube[1]}')
# if cube[1]:
#     print(f'Diameter: {cube[2]}')

In [ ]:
from time import time
start = time()
falses = 0
count = 0
print('starting')
itertime = time()
for array, cancer in cubes:
    count += 1
    print (f'cube {count}')
    if not cancer:
        falses += 1
        # plot_one_scan(array)
    newtime = time()
    print('Time between: ', newtime - itertime)
    itertime = newtime
stop = time()
print(f'Completed in {stop-start} seconds.')

In [3]:
from tensorflow.keras import Sequential, optimizers
from tensorflow.keras.layers import Conv3D, MaxPooling3D, BatchNormalization, Flatten, Dense, Dropout, ReLU

In [4]:
train_generator = generate_cube_batch(
    locations = df_annotations,
    target_size = (25,64,64),
    root_dir = 'data/train',
    batch_size = 16,
    rotate=30,
    axes=(0,1,2),
    offset=0.25
)

test_generator = generate_cube_batch(
    locations = df_annotations,
    target_size = (25,64,64),
    root_dir = 'data/test',
    batch_size = 16
)

val_generator = generate_cube_batch(
    locations = df_annotations,
    target_size = (25,64,64),
    root_dir = 'data/val',
    batch_size = 16
)

In [5]:
model = Sequential()
model.add(Conv3D(
        filters = 32,
        kernel_size = (5,5,5),
        strides = (1,1,1),
        padding = 'valid',
        activation = None,
        use_bias = True,
        kernel_initializer = 'glorot_uniform',
        bias_initializer = 'zeros',
        kernel_regularizer = None,
        bias_regularizer = None,
        activity_regularizer = None,
        kernel_constraint = None,
        bias_constraint = None,
        input_shape = (25,64,64,1)))
model.add(MaxPooling3D(
        pool_size = (2,2,2),
        strides = None,
        padding = 'valid'))
model.add(BatchNormalization(axis = 1))
model.add(ReLU(negative_slope=0.001))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(
        units = 256,
        activation = 'relu',
        use_bias = True,
        kernel_initializer = 'glorot_uniform',
        bias_initializer = 'zeros',
        kernel_regularizer = None,
        bias_regularizer = None,
        activity_regularizer = None,
        kernel_constraint = None,
        bias_constraint = None))
model.add(Dense(
        units = 32,
        activation = 'relu',
        use_bias = True,
        kernel_initializer = 'glorot_uniform',
        bias_initializer = 'zeros',
        kernel_regularizer = None,
        bias_regularizer = None,
        activity_regularizer = None,
        kernel_constraint = None,
        bias_constraint = None))
model.add(Dense(
        units = 1,
        activation = 'sigmoid',
        use_bias = True,
        kernel_initializer = 'glorot_uniform',
        bias_initializer = 'zeros',
        kernel_regularizer = None,
        bias_regularizer = None,
        activity_regularizer = None,
        kernel_constraint = None,
        bias_constraint = None))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 21, 60, 60, 32)    4032      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 10, 30, 30, 32)    0         
_________________________________________________________________
batch_normalization (BatchNo (None, 10, 30, 30, 32)    40        
_________________________________________________________________
re_lu (ReLU)                 (None, 10, 30, 30, 32)    0         
_________________________________________________________________
dropout (Dropout)            (None, 10, 30, 30, 32)    0         
_________________________________________________________________
flatten (Flatten)            (None, 288000)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               7

In [6]:
sgd = optimizers.SGD(
    lr=0.01, 
    decay = 1e-5, 
    nesterov=False
)

model.compile(
    optimizer=sgd,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    x = train_generator,
    validation_data = val_generator,
    validation_steps = 25,
    epochs = 2,
    shuffle=True,
    steps_per_epoch = 75
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 75 steps, validate for 25 steps
Epoch 1/2
74/75 [============================>.] - ETA: 14s - loss: 1.2462 - accuracy: 0.7838

In [ ]:
model.save('first_model.h5')